## Hamiltonian Monte Carlo for linear regression

The derivations presented for HMC are based on: Radford M. Neal (2012). MCMC using Hamiltonian dynamics. arXiv:1206.1901v1 

In [1]:
import numpy as np
import matplotlib.pyplot as plt

<b>Likelihood and priors</b>

In linear regression, we are given a dataset $\mathcal{D}=\{(\mathbf{x}_i, y_i)\}_{i=1}^N$ and the objective is to fit a model of form $p(y|\mathbf{w}, \mathbf{x}) = \mathcal{N}(y | \mathbf{w}^T\mathbf{x} + b, \sigma^2)$ to this data.  

To simplify the notation a bit, let us define the design matrix
$$
X = \begin{pmatrix}
x_1^1 & x_1^2 & ... & x_1^d & 1\\
x_2^1 & x_2^2 & ... & x_2^d & 1\\
. & .\\
. & .\\
. & .\\
x_N^1 & x_N^2 & ... & x_N^d & 1\\
\end{pmatrix}$$  

Let us also collect all parameters for the mean and the outputs into vectors $\boldsymbol{\theta} = [w_1, w_2, ..., w_d, b]^T$ and $\mathbf{y} = [y_1, y_2, ..., y_N]^T$  

Then
  
$$\mathbf{X}\boldsymbol{\theta} = \begin{pmatrix}
\mathbf{w}^T\mathbf{x}_1 + b\\
\mathbf{w}^T\mathbf{x}_2 + b\\
.\\
.\\
.\\
\mathbf{w}^T\mathbf{x}_N + b
\end{pmatrix}$$

We next define noninformative priors for all model parameters. For the parameters $\boldsymbol{\theta}$, the support is $\mathbb{R}^{d+1}$. A natural choice is therefore $p(\boldsymbol{\theta}) = \mathcal{N(\mathbf{0}, \sigma_0^2\mathbf{I})}$, where the variance $\sigma_0^2$ is large.

We place the prior of the noise over the precision $\tau = \sigma^{-2}$. The support is $[0, \infty)$, so a natural choice is $p(\tau) = \Gamma(\alpha_0, \beta_0)$

<b>Posterior</b>

The posterior distribution is $$p(\boldsymbol{\theta}, \tau | \mathcal{D}) = \frac{p(\boldsymbol{\theta}, \tau)p(\mathbf{y} | \mathbf{X}, \boldsymbol{\theta}, \tau)}{Z} \propto \exp(-\frac{1}{2\sigma_0^2}\boldsymbol{\theta}^T \boldsymbol{\theta}) \tau^{\alpha_0 - 1} \exp(-\beta_0\tau) \prod_{i=1}^N \sqrt{\frac{\tau}{2\pi}}\exp(-\frac{\tau}{2}(y_i - \mathbf{w}^T\mathbf{x}_i + b)^2)$$

In the likelihood, we can take the exponentiation outside the product and then combine terms which gives

$$p(\boldsymbol{\theta}, \tau | \mathcal{D}) \propto \tau^{\alpha_0 + N/2 - 1} \exp(\sum_{i=1}^N (-\frac{\tau}{2}(y_i - \mathbf{w}^T\mathbf{x}_i + b)^2) - \frac{1}{2\sigma_0^2}\boldsymbol{\theta}^T \boldsymbol{\theta} - \beta_0\tau) = \tau^{\alpha_0 + N/2 - 1} \exp(-\frac{\tau}{2}(\mathbf{y} - \mathbf{X}\boldsymbol{\theta})^T(\mathbf{y} - \mathbf{X}\boldsymbol{\theta}) - \frac{1}{2\sigma_0^2}\boldsymbol{\theta}^T \boldsymbol{\theta} - \beta_0\tau)$$

<b>Conditional distributions</b>

The posterior above does not resemble any known distribution, so we will use MCMC to sample from the posterior. We will use a Gibbs sampling scheme that alternates between sampling from the conditional distributions $p(\boldsymbol{\theta}|\tau, \mathcal{D})$ and $p(\tau|\boldsymbol{\theta}, \mathcal{D})$. For sampling the weights and the bias from $p(\boldsymbol{\theta}|\tau, \mathcal{D})$, we will use HMC. However, HMC is not optimal for sampling the precision from $p(\tau|\boldsymbol{\theta}, \mathcal{D})$, since the support is constrained to be positive. The prior we have chosen for $\tau$ is conjugate to the gaussian likelihood, so we conveniently get

$$p(\tau|\boldsymbol{\theta}, \mathcal{D}) \propto \tau^{\alpha_0 + N/2 - 1}\exp(-\tau(\frac{1}{2}(\mathbf{y} - \mathbf{X}\boldsymbol{\theta})^T(\mathbf{y} - \mathbf{X}\boldsymbol{\theta}) + \beta_0)) $$

We can now recognize that this has the form of a gamma distribution. The parameters are given by

$$\alpha - 1 = \alpha_0 + N/2 - 1 \Rightarrow \alpha = \alpha_0 + N/2$$
$$\beta = \beta_0 + \frac{1}{2}(\mathbf{y} - \mathbf{X}\boldsymbol{\theta})^T(\mathbf{y} - \mathbf{X}\boldsymbol{\theta})$$

We can conclude that $p(\tau|\boldsymbol{\theta}, \mathcal{D}) = \Gamma(\alpha, \beta)$. <i>Note:</i> The reason why we obtain the exact distribution despite dropping constants is that we only need to match the factors that depend on $\tau$, since then the normalization constants will also match as both distributions must integrate to 1.

Next, we are going to derive the unnormalized distribution for $\boldsymbol{\theta}$, which we will then sample from with HMC.

$$p(\boldsymbol{\theta} | \tau, \mathcal{D}) \propto \exp(-\frac{1}{2}(\tau(\mathbf{y} - \mathbf{X}\boldsymbol{\theta})^T(\mathbf{y} - \mathbf{X}\boldsymbol{\theta}) + \frac{1}{\sigma_0^2}\boldsymbol{\theta}^T \boldsymbol{\theta}))$$

<b>Energy functions and gradients</b>

In HMC, the energy (or Hamiltonian) $H(\mathbf{s})$of a state $\mathbf{s}$ is related to its probability by the canonical ensemble

$$p(\mathbf{s}) = \frac{1}{Z}\exp(-H(\mathbf{s})/T)$$

where $T$ is a temperature parameter. $T=1$ corresponds to our distribution of interest. Larger values of $T$ correspond to a more coarse-grained view of the distribution, which can make it easier for the chain to jump between modes. Let us assume from now on that $T=1$.

The state $\mathbf{s}$ consists of "position" variables $\mathbf{q}$ and "momentum" variables $\mathbf{p}$, i.e. $\mathbf{s} = (\mathbf{q}, \mathbf{p})$. The position variables are our original model parameters (so let us denote $\mathbf{s} = (\boldsymbol{\theta}, \mathbf{p})$ from now on), and the momentum variables are artificially introduced auxiliary variables.

The energy decomposes as a sum of potential energy $U$ and kinetic energy $K$. The potential energy is a function of position $\mathbf{q}$; it is directly related to our distribution of interest through the canonical ensemble.

$$p(\boldsymbol{\theta} | \tau, \mathcal{D}) = \frac{1}{Z}\exp(-\frac{1}{2}(\tau(\mathbf{y} - \mathbf{X}\boldsymbol{\theta})^T(\mathbf{y} - \mathbf{X}\boldsymbol{\theta}) + \frac{1}{\sigma_0^2}\boldsymbol{\theta}^T \boldsymbol{\theta})) = \frac{1}{Z}\exp(-U(\boldsymbol{\theta})) \Rightarrow U(\boldsymbol{\theta}) = \frac{1}{2}(\tau(\mathbf{y} - \mathbf{X}\boldsymbol{\theta})^T(\mathbf{y} - \mathbf{X}\boldsymbol{\theta}) + \frac{1}{\sigma_0^2}\boldsymbol{\theta}^T \boldsymbol{\theta})$$

For HMC we need the gradient of the Hamiltonian with respect to $\boldsymbol{\theta}$. For a more complicated distribution, this could be done with automatic differentiation. In our case it is quite simple, so we do it analytically.

$$\nabla_{\boldsymbol{\theta}}H(\boldsymbol{\theta}) = \nabla_{\boldsymbol{\theta}}p(\boldsymbol{\theta} | \tau, \mathcal{D}) = \frac{1}{2}(\tau\nabla_{\boldsymbol{\theta}}(\mathbf{y}^T\mathbf{y} - 2\boldsymbol{\theta}^T\mathbf{X}^T\mathbf{y} + \boldsymbol{\theta}^T\mathbf{X}^T\mathbf{X}\boldsymbol{\theta}) + \frac{1}{\sigma_0^2}\nabla_{\boldsymbol{\theta}}(\boldsymbol{\theta}^T\boldsymbol{\theta})) = \frac{1}{2}(\tau(-2\mathbf{X}^T\mathbf{y} + 2\mathbf{X}^T\mathbf{X}\boldsymbol{\theta}) + \frac{1}{\sigma_0^2}(2\boldsymbol{\theta})) = \tau(\mathbf{X}^T\mathbf{X}\boldsymbol{\theta} - \mathbf{X}^T\mathbf{y}) + \frac{1}{\sigma_0^2}\boldsymbol{\theta}$$

For the kinetic energy, we have 

$$K(\mathbf{p}) = \sum_{i=1}^d\frac{p_i^2}{2m_i}$$

which follows from the familiar formula $K = \frac{1}{2}mv^2$. The masses $m_i$ are a hyperparameters of the algorithm. The gradient of the Hamiltonian with respect to the momemtum is then

$$\nabla_{\mathbf{p}}H(\mathbf{p}) = \nabla_{\mathbf{p}}K(\mathbf{p}) = [\frac{p_1}{m_1}, \frac{p_2}{m_2}, ..., \frac{p_d}{m_d}]$$

<b>Hamiltonian mechanics</b>

In HMC, the proposal is computed by simulating Hamiltonian mechanics. This is done by solving the following ODE:

$$\frac{d\mathbf{q}}{dt} = \nabla_{\mathbf{p}}H(\mathbf{p}) = [\frac{p_1}{m_1}, \frac{p_2}{m_2}, ..., \frac{p_d}{m_d}]$$

$$\frac{d\mathbf{p}}{dt} = -\nabla_{\mathbf{q}}H(\mathbf{q}) = -\tau(\mathbf{X}^T\mathbf{X}\boldsymbol{\theta} - \mathbf{X}^T\mathbf{y}) - \frac{1}{\sigma_0^2}\boldsymbol{\theta}$$

In practise, we do this with a numerical ODE solver. The proposal is then accepted or rejected according to the Metropolis-Hastings acceptance rule.

In [ ]:
class ODESolver:
    
    def __init__(self, grad_U, epsilon, L, m):
        self.grad_U = grad_U
        self.epsilon = epsilon
        self.L = L
        self.m = m
        self.d = d
        
    def dq(self, p):
        return p / m
    
    def euler(self, q, p):
        for i in range(self.L):
            p_new = p - epsilon * self.grad_U(q)
            q = q + self.epsilon * self.dq(p)
            p = p_new
        return q, p
    
    def modified_euler(self, q, p):
        for i in range(self.L):
            p = p - self.epsilon * self.grad_U(q)
            q = q + self.epsilon * self.dq(p)
        return q, p
    
    def leapfrog(self, q, p):
        p = p - self.epsilon * self.grad_U(q) / 2
        for i in range(self.L):
            q = q + self.epsilon * self.dq(p)
            if i != self.L - 1:
                p = p - self.epsilon * self.grad_U(q)
        p = p - self.epsilon * self.grad_U(q) / 2
        p = -p
        return q, p

class HMC:
    
    def __init__(self, U, grad_U):
        
        self.U = U
        self.grad_U = grad_U
    
    def step(self):
        pass
    
    def run(self):
        pass